In [3]:
#!pip install awswrangler

In [10]:
import awswrangler as wr
import pandas as pd
import datetime as dt
import boto3
from io import StringIO

In [21]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check_gp'

In [42]:
daily_df = wr.athena.read_sql_table(
    table=table_name,
    database=database_name,
)

In [43]:
#Date format
daily_df['date'] = pd.to_datetime(daily_df['date'])

In [44]:
daily_df.head()

,payer,country,date,tx,amount,coupon_count,gp,id_main_branch,id_country,day
0,EASY PAGOS,ECUADOR,2021-11-12,108,56442.5000,2,80.6650000,T259,ECU,2021-11-12
1,ELEKTRA (MEXICO),MEXICO,2021-12-05,15463,6845500.1400,661,24671.1514000,T232,MEX,2021-12-05
2,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,2021-07-06,668,345602.7100,295,4313.6760000,T252,MEX,2021-07-06
3,BANCO AGRICOLA,EL SALVADOR,2021-07-06,743,240005.0000,314,1259.4600000,T053,ELS,2021-07-06
4,CREDOMATIC (BTS),NICARAGUA,2023-02-25,69,28919.0000,1,136.1500000,T302,NIC,2023-02-25


In [35]:
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path_pred = 'ABTv3/Tableau/' # Folder under analysis

elements = client.list_objects(Bucket=bucket_name, Prefix=path_pred)

# Initialize an empty list 
dfs = []
# Iterate through each CSV in the S3 path
for obj in elements.get('Contents', []):
#    print(obj)
    # Check if the object is a CSV file
    if obj['Key'].endswith('.csv'):
        # Read CSV content from S3
        response = client.get_object(Bucket=bucket_name, Key=obj['Key'])
        print(obj['Key'])
        csv_content = response['Body'].read().decode('utf-8')

        # Transform CSV content to df
        df = pd.read_csv(StringIO(csv_content))

        dfs.append(df)
pred_df = pd.concat(dfs, ignore_index=True)

ABTv3/Tableau/predict_2d.csv


In [48]:
#Date format
pred_df['processing_date'] = pd.to_datetime(pred_df['processing_date'])
pred_df['pred_date'] = pd.to_datetime(pred_df['pred_date'])

In [36]:
pred_df.head()

,processing_date,pred_date,pred,payer_country,model,id_main_branch,id_country,payer,country
0,2024-03-21,2023-09-08,3.372540e+06,BANCO INDUSTRIAL_GUATEMALA,MODEL_2d_BANCO INDUSTRIAL_GUATEMALA.pkl,T085,GUA,BANCO INDUSTRIAL,GUATEMALA
1,2024-03-21,2023-09-09,4.034526e+06,BANCO INDUSTRIAL_GUATEMALA,MODEL_2d_BANCO INDUSTRIAL_GUATEMALA.pkl,T085,GUA,BANCO INDUSTRIAL,GUATEMALA
2,2024-03-21,2023-09-08,7.579385e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_2d_BANCOPPEL (APPRIZA)_MEXICO.pkl,T239,MEX,BANCOPPEL (APPRIZA),MEXICO
3,2024-03-21,2023-09-09,7.809040e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_2d_BANCOPPEL (APPRIZA)_MEXICO.pkl,T239,MEX,BANCOPPEL (APPRIZA),MEXICO
4,2024-03-21,2023-09-08,4.733540e+06,BANRURAL (RYT)_GUATEMALA,MODEL_2d_BANRURAL (RYT)_GUATEMALA.pkl,T228,GUA,BANRURAL (RYT),GUATEMALA


In [20]:
mape_hist = 9 # Acá ahora pongo cualquier nro y después hay que calcularlo

In [62]:
df_final = pd.merge(daily_df, pred_df, left_on=['payer', 'country', 'id_main_branch', 'id_country', 'date'], 
                     right_on=['payer', 'country', 'id_main_branch', 'id_country', 'pred_date'],
                     how='inner')

In [63]:
# DAILY MAPE
# Convert 'amount' column to float
df_final['amount'] = df_final['amount'].astype(float)

# Calculate the absolute error between the 'amount' and 'pred' columns
df_final['error_abs'] = abs(df_final['amount'] - df_final['pred'])

# Calculate the absolute percentage error by dividing the absolute error by 'amount' and multiplying by 100
df_final['mape'] = (df_final['error_abs'] / df_final['amount']) * 100

In [64]:
df_final[['date','payer', 'country', 'amount', 'pred', 'error_abs', 'mape']]

,date,payer,country,amount,pred,error_abs,mape
0,2023-09-09,ELEKTRA (MEXICO),MEXICO,13751466.80,1.248543e+07,1.266040e+06,9.206582
1,2023-09-09,BANCO INDUSTRIAL,GUATEMALA,4006813.98,4.034526e+06,2.771245e+04,0.691633
2,2023-09-09,BBVA - BANCOMER (BTS),MEXICO,1286846.58,1.076527e+06,2.103194e+05,16.343784
3,2023-09-09,BANRURAL (RYT),GUATEMALA,6015822.24,5.681799e+06,3.340228e+05,5.552405
4,2023-09-08,ELEKTRA (MEXICO),MEXICO,12387054.70,1.161843e+07,7.686275e+05,6.205087
5,2023-09-08,BANCO INDUSTRIAL,GUATEMALA,3288016.08,3.372540e+06,8.452423e+04,2.570676
6,2023-09-08,BANCOPPEL (APPRIZA),MEXICO,8238946.57,7.579385e+06,6.595620e+05,8.005417
7,2023-09-08,BANRURAL (RYT),GUATEMALA,5112586.73,4.733540e+06,3.790463e+05,7.413983
8,2023-09-08,BBVA - BANCOMER (BTS),MEXICO,1306962.90,1.173953e+06,1.330099e+05,10.177023
9,2023-09-09,BANCOPPEL (APPRIZA),MEXICO,8845345.24,7.809040e+06,1.036305e+06,11.715827
